In [1]:
import cv2
import numpy as np
import face_recognition
import os
from datetime import datetime

In [19]:
path = "ImagesAttendance"
images = []
classNames = []
myList = os.listdir(path)

#Write The Name in ImagesAttendance Folder
for cl in myList:
    curImg = cv2.imread(f'{path}/{cl}')
    images.append(curImg)
    classNames.append(os.path.splitext(cl)[0])
print(classNames)

['Bill Gates', 'Elon Musk', 'Jack Ma', 'Prawit W', 'Prayut C', 'Steve Jobs']


In [23]:
#Create Function for Encoding
def findEncoding(images):
    encodeList = []
    for img in images:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        encode = face_recognition.face_encodings(img)[0]
        encodeList.append(encode)
    return encodeList

#Create Attendance Function
def attendance(name):
    with open('Attendance.csv','r+') as f:
        dataList = f.readlines()
        nameList = []
        for line in dataList:
            entry = line.split(',')
            nameList.append(entry[0])
        if name not in nameList:
            now = datetime.now()
            dateString = now.strftime('%Y-%m-%d')
            timeString = now.strftime('%H:%M:%S')
            f.writelines(f'\n{name},{dateString},{timeString}')
            

encodeListKnown = findEncoding(images)
print("Encoding Completed")

#Create Webcam
cap = cv2.VideoCapture(0)

while True:
    success, img = cap.read()
    imgS = cv2.resize(img, (0,0), None, 0.25,0.25) # for fast processing
    imgS = cv2.cvtColor(imgS, cv2.COLOR_BGR2RGB)
    
    #Find Face Location & Encoding
    facesCurFrame = face_recognition.face_locations(imgS) 
    encodesCurFrame = face_recognition.face_encodings(imgS, facesCurFrame)
    
    for encodeFace, faceLoc in zip(encodesCurFrame,facesCurFrame):
        matches = face_recognition.compare_faces(encodeListKnown, encodeFace)
        faceDis = face_recognition.face_distance(encodeListKnown, encodeFace)
        print(faceDis)
        matchIndex = np.argmin(faceDis)
        
        if matches[matchIndex]:
            name = classNames[matchIndex].upper()
            print(name)
            
            #Face Location
            y1, x2, y2, x1 = faceLoc
            
            #Return Face Location to Original Size
            y1, x2, y2, x1 = y1*4, x2*4, y2*4, x1*4
            
            #Draw Rectangle & Put Text
            cv2.rectangle(img, (x1, y1), (x2,y2), (0,255,0), 2)
            cv2.rectangle(img, (x1, y2-35), (x2,y2), (0,255,0),cv2.FILLED)
            cv2.putText(img, name, (x1+6, y2-6), cv2.FONT_HERSHEY_COMPLEX, 0.5, (255,255,255),2)
            
            #Call attendance function
            attendance(name)
            
    #Show Webcam        
    cv2.imshow("Webcam", img)
    
    wait_time = 1
    if cv2.waitKey(wait_time) & 0xFF == ord('q'):
        cv2.destroyAllWindows()
        break

Encoding Completed
[0.82342816 0.75543941 0.58540276 0.55350683 0.37346175 0.9600038 ]
PRAYUT C
[0.80581983 0.74470203 0.58728386 0.56301339 0.39418895 0.94737158]
PRAYUT C
[0.81206743 0.75514435 0.5928347  0.5871869  0.40491037 0.94806824]
PRAYUT C
[0.85212999 0.7671164  0.5537683  0.53769071 0.35226126 0.95417854]
PRAYUT C
[0.80939055 0.75182793 0.57830984 0.57789925 0.40344068 0.95300304]
PRAYUT C
[0.83444405 0.75955244 0.56530052 0.54576894 0.35666027 0.94005304]
PRAYUT C
[0.83168297 0.76342465 0.56968004 0.56462502 0.37153679 0.96509861]
PRAYUT C
[0.84563011 0.56417183 0.86174103 0.83663853 0.73868337 0.88648588]
ELON MUSK
[0.82871473 0.53309735 0.79342862 0.77327634 0.70195278 0.88278131]
ELON MUSK
[0.81543446 0.4912992  0.82376876 0.76603255 0.72954856 0.88443036]
ELON MUSK
[0.85240826 0.54313501 0.7872471  0.73732913 0.69248025 0.89593458]
ELON MUSK
[0.83545902 0.51425491 0.83033076 0.76060949 0.72487731 0.90455555]
ELON MUSK
[0.6154849  0.85020766 0.87393024 0.91840754 0.79251